In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from reinforcement.players import DummyPlayer, ModelPlayer
from reinforcement.sequential_dqn.dqn_v2 import DQNTrainerV2
from modeling.chessformer import ChessFormer
from torch.optim import AdamW
import torch as t
from copy import deepcopy

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BUFFER_SIZE = 1_000_000
RANDOM_ACTION_RATE = 0.0
BATCH_SIZE = 32
NB_STEPS_REWARD = 1
WARM_UP_STEPS = 10_000
ALPHA_SAMPLING = 0.7
BETA_SAMPLING = 0.5

In [4]:
#model_1 = ChessFormer(6, 3, 512, 2048, 64, 8, 0.0).to('cuda')
#model_2 = ChessFormer(6, 3, 512, 2048, 64, 8, 0.0).to('cuda')
##model_1_state_dict = t.load('runs/experiment_4/model_1.pt', map_location='cuda')
##model_1.load_state_dict(model_state_dict)
#optimizer_1 = AdamW(model_1.parameters(), lr=1e-3, eps=1e-2)
#optimizer_2 = AdamW(model_2.parameters(), lr=1e-3, eps=1e-2)
#competitor = DummyPlayer()
##competitor_model = deepcopy(model)
##competitor_model.requires_grad_(False)
##competitor = ModelPlayer(competitor_model, random_action_rate=0.0, model_device='cuda')

In [5]:
model_1 = ChessFormer(6, 3, 512, 2048, 64, 8, 0.0).to('cuda')
model_2 = ChessFormer(6, 3, 512, 2048, 64, 8, 0.0).to('cuda')
#model_1_state_dict = t.load('runs/experiment_4/model_1.pt', map_location='cuda')
#model_1.load_state_dict(model_state_dict)
optimizer = AdamW(model_1.parameters(), lr=1e-4, eps=1e-1)
competitor = DummyPlayer()

In [6]:
TAU = 0.0001

In [7]:
#trainer = DQNTrainerV2(model_1,
#                       model_2,
#                       optimizer_1,
#                       optimizer_2,
#                       BUFFER_SIZE,
#                       competitor,
#                       BATCH_SIZE,
#                       experiment_name='experiment_11',
#                       models_device='cuda',
#                       nb_steps_reward=NB_STEPS_REWARD,
#                       warm_up_steps=WARM_UP_STEPS,
#                       alpha_sampling=ALPHA_SAMPLING,
#                       beta_sampling=BETA_SAMPLING)

In [8]:
trainer = DQNTrainerV2(model_1=model_1,
                       model_2=model_2,
                       optimizer=optimizer,
                       buffer_size=BUFFER_SIZE,
                       competitor=competitor,
                       batch_size=BATCH_SIZE,
                       experiment_name='experiment_15',
                       model_device='cuda',
                       nb_steps_reward=NB_STEPS_REWARD,
                       warm_up_steps=WARM_UP_STEPS,
                       alpha_sampling=ALPHA_SAMPLING,
                       beta_sampling=BETA_SAMPLING,
                       tau=TAU)

2023-03-28 08:49:31.837851: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 08:49:32.560391: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-28 08:49:32.560439: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-28 08:49:32.560443: W 

In [9]:
trainer.train(15000)

 48%|████▊     | 7214/15000 [31:15:09<45:40:14, 21.12s/it]

In [ ]:
t.save(model_1.state_dict(), 'runs/experiment_15/model.pt')
#t.save(model_2.state_dict(), 'runs/experiment_11/model2.pt')